In [152]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup  
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)



In [ ]:
df=pd.read_excel(f'{f}', sheet_name=0, header=0, usecols='A,AJ,CZ:DI',index_col=0).reset_index()
for c in df.columns:
    if 'Unnamed' in c:
        df.drop(columns=c, inplace=True)
df.applymap(lambda x: re.sub('[^\x00-\x7F]','',str(x)) )

In [ ]:

converts={
    'Selections':lambda x: re.sub('\d{,2}\.','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{f}', sheet_name=0, header=0, usecols='A:X',index_col=0).reset_index()
df


In [156]:
with open(ctx.get_tempalte_dir()+'temp.aspx', 'r') as f: 
    t=f.read() 
    for c in df.columns: 
        print(t.replace("{col}",c.strip()).replace("{icn}",c.strip())  ) 

<CB:DataField  DBColumnName="Agency Name" ImportColumnName="Agency Name"  runat="server"/> 
<CB:DataField  DBColumnName="Key Length" ImportColumnName="Key Length"  runat="server"/> 
<CB:DataField  DBColumnName="On Premises" ImportColumnName="On Premises"  runat="server"/> 
<CB:DataField  DBColumnName="Commercial - Amazon Web Services (AWS)" ImportColumnName="Commercial - Amazon Web Services (AWS)"  runat="server"/> 
<CB:DataField  DBColumnName="Cloud Service Offering" ImportColumnName="Cloud Service Offering"  runat="server"/> 
<CB:DataField  DBColumnName="Hybrid Cloud Providers.1" ImportColumnName="Hybrid Cloud Providers.1"  runat="server"/> 
<CB:DataField  DBColumnName="FIPS 199" ImportColumnName="FIPS 199"  runat="server"/> 


In [ ]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists   WHERE PK_PickListType > 410 ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==427]['Selections'])
print(s) 

In [ ]:
base = r"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\HVA\\2023\\"
pages = {
       '3':'4045','4':'4046','5':'4047' 
} 
src = r'C:\\_som\\tmp\\'
for i in pages.keys():
    exts = ['.aspx','.aspx.designer.vb','.aspx.vb']
    for ext in exts:
        with open(f'{src}2023_HVA_MonitoringAndDetection_1{ext}', 'r', encoding='UTF-8') as fr:
            txt = fr.read()
            txt=txt.replace(f'11',f'{i}{i}')
            txt=txt.replace(f'_1',f'_{i}')
            txt=txt.replace(f'"4043"',f'"{pages[i]}"')
            df, code = generate_code_from_db(ctx, qgroup=pages[i])
            txt=txt.replace(f'<!--controls-->',code)
            #print(txt)
        with open(f'{base}2023_HVA_MonitoringAndDetection_{i}{ext}', 'w', encoding='UTF-8') as fw:
            fw.write(txt)

In [153]:
ip = issue_provider(ctx)  
ip.driver.quit()
#ip.issues
print(ip.issues.to_string())



====== WebDriver manager ======
Current google-chrome version is 110.0.5481
Get LATEST driver version for 110.0.5481
There is no [win32] chromedriver for browser 110.0.5481 in cache
Get LATEST driver version for 110.0.5481
Trying to download new driver from https://chromedriver.storage.googleapis.com/110.0.5481.77/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\timko\.wdm\drivers\chromedriver\win32\110.0.5481.77]


        Key                                                                                                                 Summary
0   CS-9449                                                  Post Quantum Cryptography (PQC) Data Call 2023 - Admin Approval/Reopen
1   CS-9448                                                 Post Quantum Cryptography (PQC) Data Call 2023 - User Submission/Reopen
2   CS-9446                                                   Post Quantum Cryptography (PQC) Data Call 2023 - Validate and Approve
3   CS-9444                                    Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Upload/Download Feature
4   CS-9443                                                       Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Form
5   CS-9442                                            Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Database Script
6   CS-9432                                                                 

In [ ]:
ip.driver.quit()

In [154]:
print(ip.issues.to_string())

        Key                                                                                                                 Summary
0   CS-9449                                                  Post Quantum Cryptography (PQC) Data Call 2023 - Admin Approval/Reopen
1   CS-9448                                                 Post Quantum Cryptography (PQC) Data Call 2023 - User Submission/Reopen
2   CS-9446                                                   Post Quantum Cryptography (PQC) Data Call 2023 - Validate and Approve
3   CS-9444                                    Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Upload/Download Feature
4   CS-9443                                                       Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Form
5   CS-9442                                            Post Quantum Cryptography (PQC) Data Call 2023 - Section 1 - Database Script
6   CS-9432                                                                 

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 

In [ ]:
df,code=generate_code_from_db(ctx,4019) 
#generate_section(4018, code)
df[:10]
#sql=df_to_sql(df,ctx,template_name='fsma_QuestionsInsert.sql')

In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]